In [1]:
#https://scrapfly.io/blog/web-scraping-with-selenium-and-python/
# https://stackoverflow.com/questions/53475578/python-selenium-multiprocessing

TODO
- явно есть ошибки не корретного парсинга, можно найти кейсы по дешевым билетам
- подумать может сделать без селениума
- на сервер запустить скрипт

In [1]:
import os
import json
import asyncio
from datetime import datetime, timedelta
from concurrent.futures.thread import ThreadPoolExecutor

import nest_asyncio
from utils.aviasales_parser import parse_page, list_dates, get_ticket

nest_asyncio.apply()

In [2]:
from itertools import product
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")



In [3]:
def scrape(origin, destination, datetime_object, *, loop):
    loop.run_in_executor(executor, get_ticket, origin, destination, datetime_object)

In [4]:
executor = ThreadPoolExecutor(3)
loop = asyncio.get_event_loop()


origins =  ['BUD', 'TSR', 'OSI', 'PEV', 'SOB', 'DEB', 'ZAG', 'BEG', 'TGD'] #['LED', 'MOW', 'KZN']
destinations = ['HEL', 'TLL']# ['BEG', 'TGD', 'TIA', 'TIV']
range_date = ['2023-11-01', '2023-11-15']
number_ticket = 1

date_start, date_end = range_date[0], range_date[1]
datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')
range_dates = list_dates(datetime_start, datetime_end)

parameters = list(product(origins, destinations, list_dates(datetime_start, datetime_end)))

# билеты туда
for origin, destination, datetime_object in tqdm(parameters, total=len(parameters), desc='parse ticket'):
    scrape(origin, destination, datetime_object, loop=loop)



parse ticket:   0%|          | 0/270 [00:00<?, ?it/s]

In [3]:
# # чтобы обработать все json
# root = '.\\tickets\\json'
# json_pattern = os.path.join(root,'*.json')
# file_list = glob.glob(json_pattern)

# dfs = [] # an empty list to store the data frames
# for file in file_list:
#     data = pd.read_json(file) # read data frame from json file
#     dfs.append(data) # append the data frame to the list

# df = pd.concat(dfs, ignore_index=True) 
# df.to_csv('tickets.txt', sep = ';', index = False, encoding='utf-8-sig')

In [ ]:
root_json = '.\\tickets\\json'
json_pattern = os.path.join(root_json,'*.json')
file_list = glob.glob(json_pattern)
ticket_list = list(map(lambda)